In [1]:
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip

--2025-02-17 19:23:23--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
200 OKequest sent, awaiting response... 
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

100%[======================================>] 4,924,029   12.1MB/s   in 0.4s   

2025-02-17 19:23:24 (12.1 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base      

In [2]:
%cd ml-100k
!shuf ua.base -o ua.base.shuffled
!head -10 ua.base.shuffled

/home/ec2-user/SageMaker/ml-100k
481	505	5	885828574
707	283	4	880059957
299	895	2	884993860
940	628	4	885921800
407	25	3	876339975
916	393	2	880845067
57	844	2	883697134
495	210	5	888632496
851	696	3	874728338
899	474	3	884121612


In [3]:

import sagemaker
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role
from sagemaker.deserializers import JSONDeserializer

import boto3, csv, io, json
import numpy as np
from scipy.sparse import lil_matrix

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/17/25 19:23:27] INFO     Found credentials from IAM Role:                                   ]8;id=284024;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=690698;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [4]:
nbUsers=943
nbMovies=1682
nbFeatures=nbUsers+nbMovies

nbRatingsTrain=90570
nbRatingsTest=9430


In [5]:
def loadDataset(filename, lines, columns):
    # Features are one-hot encoded in a sparse matrix
    X = lil_matrix((lines, columns)).astype('float32')
    # Labels are stored in a vector
    Y = []
    line=0
    with open(filename,'r') as f:
        samples=csv.reader(f,delimiter='\t')
        for userId,movieId,rating,timestamp in samples:
            X[line,int(userId)-1] = 1
            X[line,int(nbUsers)+int(movieId)-1] = 1
            if int(rating) >= 4:
                Y.append(1)
            else:
                Y.append(0)
            line=line+1
            
    Y=np.array(Y).astype('float32')
    return X,Y


In [6]:
X_train, Y_train = loadDataset('ua.base.shuffled', nbRatingsTrain, nbFeatures)
X_test, Y_test = loadDataset('ua.test',nbRatingsTest,nbFeatures)


In [7]:
print(X_train.shape)
print(Y_train.shape)
assert X_train.shape == (nbRatingsTrain, nbFeatures)
assert Y_train.shape == (nbRatingsTrain, )
zero_labels = np.count_nonzero(Y_train)
print("Training labels: %d zeros, %d ones" % (zero_labels, nbRatingsTrain-zero_labels))

print(X_test.shape)
print(Y_test.shape)
assert X_test.shape  == (nbRatingsTest, nbFeatures)
assert Y_test.shape  == (nbRatingsTest, )
zero_labels = np.count_nonzero(Y_test)
print("Test labels: %d zeros, %d ones" % (zero_labels, nbRatingsTest-zero_labels))


(90570, 2625)
(90570,)
Training labels: 49906 zeros, 40664 ones
(9430, 2625)
(9430,)
Test labels: 5469 zeros, 3961 ones


In [8]:

bucket = 'moviesrecommendationsystembucket'
prefix = 'sagemaker/fm-movielens'

train_key      = 'train.protobuf'
train_prefix   = '{}/{}'.format(prefix, 'train')

test_key       = 'test.protobuf'
test_prefix    = '{}/{}'.format(prefix, 'test')

output_prefix  = 's3://{}/{}/output'.format(bucket, prefix)


In [9]:
def writeDatasetToProtobuf(X, Y, bucket, prefix, key):
    buf = io.BytesIO()
    smac.write_spmatrix_to_sparse_tensor(buf, X, Y)
    buf.seek(0)
    obj = '{}/{}'.format(prefix, key)
    boto3.resource('s3').Bucket(bucket).Object(obj).upload_fileobj(buf)
    return 's3://{}/{}'.format(bucket,obj)
    
train_data = writeDatasetToProtobuf(X_train, Y_train, bucket, train_prefix, train_key)    
test_data  = writeDatasetToProtobuf(X_test, Y_test, bucket, test_prefix, test_key)    
  
print(train_data)
print(test_data)
print('Output: {}'.format(output_prefix))


[02/17/25 19:23:37] INFO     Found credentials from IAM Role:                                   ]8;id=474062;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=845290;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

s3://moviesrecommendationsystembucket/sagemaker/fm-movielens/train/train.protobuf
s3://moviesrecommendationsystembucket/sagemaker/fm-movielens/test/test.protobuf
Output: s3://moviesrecommendationsystembucket/sagemaker/fm-movielens/output


In [10]:
containers = {'us-west-2': '174872318107.dkr.ecr.us-west-2.amazonaws.com/factorization-machines:latest',
              'us-east-1': '382416733822.dkr.ecr.us-east-1.amazonaws.com/factorization-machines:latest',
              'us-east-2': '404615174143.dkr.ecr.us-east-2.amazonaws.com/factorization-machines:latest',
              'eu-west-1': '438346466558.dkr.ecr.eu-west-1.amazonaws.com/factorization-machines:latest'}


In [11]:
fm = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                   get_execution_role(), 
                                   instance_count=1, 
                                   instance_type='ml.c4.xlarge',
                                   output_path=output_prefix,
                                   sagemaker_session=sagemaker.Session())

fm.set_hyperparameters(feature_dim=nbFeatures,
                      predictor_type='binary_classifier',
                      mini_batch_size=1000,
                      num_factors=64,
                      epochs=100)

fm.fit({'train': train_data, 'test': test_data})


[02/17/25 19:23:39] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=557496;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=697849;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=416487;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=823661;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             factorization-machines-2025-02-17-19-23-39-499                                        

2025-02-17 19:23:40 Starting - Starting the training job...
..25-02-17 19:23:54 Starting - Preparing the instances for training.
..............24:41 Downloading - Downloading the training image.
2025-02-17 19:26:58 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
/opt/amazon/lib/python3.8/site-packages/mxnet/model.py:97: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if num_device is 1 and 'dist' not in kvstore:
[02/17/2025 19:27:09 INFO 140435427256128] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-conf.json: {'epochs': 1, 'mini_batch_size': '1000', 'use_bias': 'true', 'use_linear': 'true', 'bias_lr': '0.1', 'linear_lr': '0.001', 'factors_lr': '0.0001', 'bias_wd': '0.01', 'linear_wd': '0.001', 'factors_wd': '0.00001', 'bias_init_method': 'normal', 'bias_init_sigma': '0.01', 'linear_init_method': 'normal',

In [12]:
fm_predictor = fm.deploy(instance_type='ml.c4.xlarge', initial_instance_count=1)

[02/17/25 19:28:28] INFO     Creating model with name:                                              ]8;id=743611;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=218676;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\
                             factorization-machines-2025-02-17-19-28-28-017                                        

                    INFO     Creating endpoint-config with name                                     ]8;id=657580;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=362603;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\
                             factorization-machines-2025-02-17-19-28-28-017                                        

                    INFO     Creating endpoint with name                                            ]8;id=583820;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=308749;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\
                             factorization-machines-2025-02-17-19-28-28-017                                        

---------!

In [77]:
import json
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

class FMSerializer(JSONSerializer):
    def serialize(self, data):
        js = {"instances": []}
        for row in data:
            js["instances"].append({"features": row.tolist()})
        return json.dumps(js)

fm_predictor.serializer = FMSerializer()
fm_predictor.deserializer = JSONDeserializer()

result = fm_predictor.predict(X_test[0:10].toarray(), initial_args={"ContentType": "application/json"})
print(result)
print(type(result))
print (Y_test[0:10])


{'predictions': [{'score': 0.6369118690490723, 'predicted_label': 1.0}, {'score': 0.49481257796287537, 'predicted_label': 0.0}, {'score': 0.6743587255477905, 'predicted_label': 1.0}, {'score': 0.5837450623512268, 'predicted_label': 1.0}, {'score': 0.23433122038841248, 'predicted_label': 0.0}, {'score': 0.5716763734817505, 'predicted_label': 1.0}, {'score': 0.7905181050300598, 'predicted_label': 1.0}, {'score': 0.8664506077766418, 'predicted_label': 1.0}, {'score': 0.7397288680076599, 'predicted_label': 1.0}, {'score': 0.7464497089385986, 'predicted_label': 1.0}]}
<class 'dict'>
[1. 1. 1. 0. 0. 1. 1. 0. 1. 1.]


## **📌 Steps to Extract Model Predictions for a User**

### **1️⃣ Generate One-Hot Encoded User-Movie Features**
- Construct a **one-hot encoded feature vector** where:
  - **User ID** is represented in the first 943 features.
  - **Movie ID** is represented in the next 1682 features.
- Create **one input row per movie** to represent the selected user's interaction with each movie.

### **2️⃣ Use SageMaker's `fm_predictor` for Batch Predictions**
- Send requests in **batches** (e.g., `100` movies per batch) to avoid exceeding SageMaker's request size limits.
- The model returns:
  - **Score** → Probability of the user liking the movie.
  - **Predicted Label** → `1` (Like) or `0` (Dislike).

### **3️⃣ Sort Movies by Predicted Score & Extract Top 10**
- **Sort predictions in descending order** based on the confidence score.
- Extract the **top 10 highest-scoring movies** as the **recommended movies**.


In [ ]:
import numpy as np
from scipy.sparse import lil_matrix

# Constants from MovieLens dataset
nbUsers = 943
nbMovies = 1682
nbFeatures = nbUsers + nbMovies

def create_user_movie_features(user_id):
    """
    Create a one-hot encoded feature matrix for all movies for a given user.

    Parameters:
    - user_id (int): User ID (1-based index)

    Returns:
    - Numpy array of feature vectors for all movies
    """
    X = lil_matrix((nbMovies, nbFeatures), dtype='float32')

    for movie_id in range(1, nbMovies + 1):
        X[movie_id - 1, user_id - 1] = 1  # One-hot encode user ID
        X[movie_id - 1, nbUsers + movie_id - 1] = 1  # One-hot encode movie ID

    return X  # Convert sparse matrix to dense array

In [79]:
# Define batch size
BATCH_SIZE = 100  

# Function to predict in batches
def get_predictions(X_data, batch_size):
    predictions = []
    for i in range(0, X_data.shape[0], batch_size):
        batch = X_data[i : i + batch_size].toarray()  # Convert sparse to dense

        # Send batch request to SageMaker endpoint
        response = fm_predictor.predict(batch, initial_args={"ContentType": "application/json"})
        # Parse JSON response
        parsed_response = response  # Ensure it's a dictionary
        
        if "predictions" in parsed_response:
            predictions.extend(parsed_response["predictions"])
        else:
            print(f"Unexpected response format: {parsed_response}")
            return []
    
    return predictions

In [86]:
# Select a user (e.g., User ID = 5)
user_id = 5

# Generate input feature vectors for all movies for the selected user
X_user = create_user_movie_features(user_id)

# Make batch predictions using fm_predictor
predictions = get_predictions(X_user, BATCH_SIZE)

# # Sort movies by highest predicted score
top_movies = sorted(enumerate(predictions), key=lambda x: x[1]['score'], reverse=True)[:10]

# # Display top 10 recommendations
print(f"Top 10 recommended movies for User {user_id}:")
for idx, pred in top_movies:
    print(f"Movie ID: {idx + 1}, Score: {pred['score']:.4f}, Predicted Like: {pred['predicted_label']}")

Top 10 recommended movies for User 5:
Movie ID: 483, Score: 0.9190, Predicted Like: 1.0
Movie ID: 127, Score: 0.8997, Predicted Like: 1.0
Movie ID: 98, Score: 0.8950, Predicted Like: 1.0
Movie ID: 479, Score: 0.8867, Predicted Like: 1.0
Movie ID: 357, Score: 0.8675, Predicted Like: 1.0
Movie ID: 427, Score: 0.8558, Predicted Like: 1.0
Movie ID: 64, Score: 0.8552, Predicted Like: 1.0
Movie ID: 511, Score: 0.8543, Predicted Like: 1.0
Movie ID: 285, Score: 0.8530, Predicted Like: 1.0
Movie ID: 318, Score: 0.8511, Predicted Like: 1.0


## **Steps to Evaluate Model Performance**

### **1️⃣ Compute Accuracy & F1 Score for Training and Testing**
- Use **batch inference** to obtain predictions for both **training (`X_train`)** and **testing (`X_test`)** datasets.
- Extract the **predicted labels (`1 = Like, 0 = Dislike`)** from the SageMaker response.
- Compute **accuracy** and **F1 score** using `sklearn.metrics`:
  - **Accuracy** → Measures the percentage of correctly classified ratings.
  - **F1 Score** → Balances precision and recall, useful for imbalanced data.

### **2️⃣ Analyze Model Performance**
- Compare **training vs. testing performance**:
  - Check for **overfitting** if training accuracy is significantly higher than testing accuracy.
  - Observe if the model generalizes well on unseen data.

In [78]:
from sklearn.metrics import accuracy_score, f1_score
import json

# Function to extract predicted labels
def extract_labels(predictions):
    """
    Extracts the predicted labels (0 or 1) from the SageMaker response.

    Parameters:
    - predictions: List of prediction dictionaries from SageMaker.

    Returns:
    - List of predicted labels.
    """
    return [int(pred["predicted_label"]) for pred in predictions]

In [80]:
train_predictions = get_predictions(X_train, BATCH_SIZE)
train_pred_labels = extract_labels(train_predictions)

# Calculate Accuracy & F1 Score
train_accuracy = accuracy_score(Y_train, train_pred_labels)
train_f1 = f1_score(Y_train, train_pred_labels)

In [82]:
print("📊 Model Evaluation Metrics:")
print(f"✅ Training Accuracy: {train_accuracy:.4f}")
print(f"✅ Training F1 Score: {train_f1:.4f}")
print(f"✅ Testing Accuracy: {test_accuracy:.4f}")
print(f"✅ Testing F1 Score: {test_f1:.4f}")

📊 Model Evaluation Metrics:
✅ Training Accuracy: 0.7502
✅ Training F1 Score: 0.7784
✅ Testing Accuracy: 0.6976
✅ Testing F1 Score: 0.7408


# 🎯 Model Evaluation Report

## 📌 a) Report Model Accuracy & F1 Score
The model's performance on both **training** and **testing** datasets is summarized below:

| **Metric**      | **Training Data** | **Testing Data** |
|----------------|----------------|----------------|
| **Accuracy**   | **0.7502**      | **0.6976**      |
| **F1 Score**   | **0.7784**      | **0.7408**      |

---

## 📌 b) Analysis of the Results

### **1️⃣ Training vs. Testing Performance**
- **Training Accuracy: 75.02%**
- **Testing Accuracy: 69.76%**
- **Training F1 Score: 77.84%**
- **Testing F1 Score: 74.08%**

🔹 **The training accuracy (75.02%) is higher than the test accuracy (69.76%)**, indicating that the model performs better on known data.  

🔹 **The F1 Score follows the same trend**, being **higher in training (77.84%) than testing (74.08%)**.  

🔹 **A ~5.26% drop in accuracy** from training to testing **suggests mild overfitting**. While the model generalizes decently, there is room for improvement.

---

### **2️⃣ Possible Reasons for Performance Drop**
📌 **Overfitting**: The model might be memorizing training patterns rather than generalizing well.  
📌 **Data Imbalance**: If the dataset has more **"liked" (1s) than "not liked" (0s)**, accuracy alone might not be a sufficient measure.  
📌 **Sparse Data**: Since user-movie interactions are one-hot encoded, sparsity can reduce predictive power.  
📌 **Factorization Machines Limitations**: FM models are great for pairwise interactions but may not capture complex patterns well. **Deep learning-based models (e.g., Neural Collaborative Filtering, Transformers) could work better.**

---